In [1]:
from sympy import diff, hessian
import numpy as np

In [ ]:
from sympy import diff, hessian


def calcula_gradiente(funcao, variaveis):
    return [diff(funcao, variavel) for variavel in variaveis]


def calcula_hessiana(funcao, variaveis):
    return hessian(funcao, variaveis)


In [ ]:
import numpy as np


def substitui_variaveis_funcao(funcao, variaveis, ponto):
    return funcao.subs(dict(zip(variaveis, ponto)))


def substitui_variaveis_gradiente(gradiente, variaveis, ponto):
    return np.array(
        [gradiente_i.subs(dict(zip(variaveis, ponto))) for gradiente_i in gradiente]
    ).astype(np.float64)


def substitui_variaveis_hessiana(hessiana, variaveis, ponto):
    return np.array(hessiana.subs(dict(zip(variaveis, ponto)))).astype(np.float64)


<img src="./img/bfgs.jpeg" width="500">


In [4]:
def calcula_bfgs(H, p, q):
    p = p.reshape(2,1)
    q = q.reshape(2,1)

    termo_2_1 = (1 + (((q.T @ H) @ q) / (p.T @ q)))
    termo_2_2 = (p @ p.T) / (p.T @ q)
    termo_2 = termo_2_1 * termo_2_2
    termo_3 = ((p @ (q.T @ H)) + ((H @ q) @ p.T)) / (p.T @ q)
    return H + termo_2 - termo_3

<img src="./img/dfp.jpeg" width="500">


In [5]:
def calcula_dfp(H, p, q):
    p = p.reshape(2,1)
    q = q.reshape(2,1)

    termo_2 = (p.T @ p) / (p.T @ q)
    termo_3 = ((H @ q) @ (q.T @ H)) / ((q.T @ H) @ q)
    return H + termo_2 - termo_3

![Armijo's Rule](./img/alg_busca_armijo.png)

In [ ]:
def busca_de_armijo(funcao: any, gradiente: any, variaveis: list, ponto_inicial: np.array, d: np.array, gama: float, eta: float):
  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - d: direção de descida
    - gama: fator de redução do passo
    - eta: fator de ???     # TODO: O que é o eta?

  Retorna:
    - t: passo ótimo
    - iteracoes: número de iterações
  """
  t = 1
  iteracoes = 1
  while substitui_variaveis_funcao(funcao, variaveis, ponto_inicial + t*d) > substitui_variaveis_funcao(funcao, variaveis, ponto_inicial) + eta*t*np.dot(substitui_variaveis_gradiente(gradiente, variaveis, ponto_inicial), d):
    t = gama*t
    iteracoes += 1

  return t, iteracoes

<h2>Método do Gradiente</h2>
<img src="./img/alg_metodo_gradiente.png" width="500">


In [ ]:
def metodo_do_gradiente(
    funcao: any,
    gradiente: any,
    variaveis: list,
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000,
    valor_minimo: float = 1e-6):

  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ??? # TODO: O que é o eta?
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução

  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial
  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp
    ponto = ponto + t*d
    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1

  return ponto, iteracoes, iteracoes_armijo

<h2>Método de Newton</h2>
<img src="./img/alg_metodo_newton.png" width="500">

In [ ]:
def metodo_de_newton(
    funcao: any,
    gradiente: any,
    hessiana: any,
    variaveis: list,
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000,
    valor_minimo: float = 1e-6):

  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - hessiana: hessiana da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???     # TODO: O que é o eta?
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução

  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial

  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -np.linalg.inv(substitui_variaveis_hessiana(hessiana, variaveis, ponto)) @ substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp
    ponto = ponto + t*d
    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1

  return ponto, iteracoes, iteracoes_armijo

<h2>Método de Quase-Newton</h2>
<img src="./img/alg_metodo_quase_newton.png" width="500">

In [ ]:
def metodo_de_quase_newton(
    funcao: any,
    n_variaveis: int,
    gradiente: any,
    variaveis: list,
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000,
    valor_minimo: float = 1e-6,
    metodo: str = 'dfp'
    ):

  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - hessiana: hessiana da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???     # TODO: O que é o eta?
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução
    - metodo: método de atualização

  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial
  H = np.eye(n_variaveis)

  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -H @ substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp

    proximo_ponto = ponto + t*d
    p = proximo_ponto - ponto
    q = substitui_variaveis_gradiente(gradiente, variaveis, proximo_ponto) - substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    ponto = proximo_ponto

    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1
    H = calcula_dfp(H, p, q) if metodo == 'dfp' else calcula_bfgs(H, p, q)

  return ponto, iteracoes, iteracoes_armijo

In [ ]:
# convert the following sum func to sympy: sum from i=1 to 100: xi**4 - 16xi**2 + 5xi

### Função 1

In [ ]:
from sympy import symbols

variaveis = list(symbols('x1:8'))
print("Variáveis da função 1:")
display(variaveis)

Variáveis da função 1:


[x1, x2, x3, x4, x5, x6, x7]

In [67]:
f_x = sum([100 * (variaveis[i+1] - variaveis[i]**2)**2 + (1 - variaveis[i])**2 for i in range(len(variaveis)-1)])
print("Função 1:")
display(f_x)

Função 1:


(1 - x1)**2 + (1 - x2)**2 + (1 - x3)**2 + (1 - x4)**2 + (1 - x5)**2 + (1 - x6)**2 + 100*(-x1**2 + x2)**2 + 100*(-x2**2 + x3)**2 + 100*(-x3**2 + x4)**2 + 100*(-x4**2 + x5)**2 + 100*(-x5**2 + x6)**2 + 100*(-x6**2 + x7)**2

In [68]:
grad_f_x = calcula_gradiente(f_x, variaveis)
print("Vetor gradiente da função 1:")
display(grad_f_x)

Vetor gradiente da função 1:


[-400*x1*(-x1**2 + x2) + 2*x1 - 2,
 -200*x1**2 - 400*x2*(-x2**2 + x3) + 202*x2 - 2,
 -200*x2**2 - 400*x3*(-x3**2 + x4) + 202*x3 - 2,
 -200*x3**2 - 400*x4*(-x4**2 + x5) + 202*x4 - 2,
 -200*x4**2 - 400*x5*(-x5**2 + x6) + 202*x5 - 2,
 -200*x5**2 - 400*x6*(-x6**2 + x7) + 202*x6 - 2,
 -200*x6**2 + 200*x7]

In [69]:
hess_f_x = calcula_hessiana(f_x, variaveis)
print("Matriz Hessiana da função 1:")
display(hess_f_x)

Matriz Hessiana da função 1:


Matrix([
[1200*x1**2 - 400*x2 + 2,                   -400*x1,                         0,                         0,                         0,                         0,       0],
[                -400*x1, 1200*x2**2 - 400*x3 + 202,                   -400*x2,                         0,                         0,                         0,       0],
[                      0,                   -400*x2, 1200*x3**2 - 400*x4 + 202,                   -400*x3,                         0,                         0,       0],
[                      0,                         0,                   -400*x3, 1200*x4**2 - 400*x5 + 202,                   -400*x4,                         0,       0],
[                      0,                         0,                         0,                   -400*x4, 1200*x5**2 - 400*x6 + 202,                   -400*x5,       0],
[                      0,                         0,                         0,                         0,                   -400*x5, 12

### Função 2

In [72]:
from sympy import symbols
variaveis = list(symbols('x1:101'))
print("Variáveis da função 2:")
display(variaveis)

Variáveis da função 2:


[x1,
 x2,
 x3,
 x4,
 x5,
 x6,
 x7,
 x8,
 x9,
 x10,
 x11,
 x12,
 x13,
 x14,
 x15,
 x16,
 x17,
 x18,
 x19,
 x20,
 x21,
 x22,
 x23,
 x24,
 x25,
 x26,
 x27,
 x28,
 x29,
 x30,
 x31,
 x32,
 x33,
 x34,
 x35,
 x36,
 x37,
 x38,
 x39,
 x40,
 x41,
 x42,
 x43,
 x44,
 x45,
 x46,
 x47,
 x48,
 x49,
 x50,
 x51,
 x52,
 x53,
 x54,
 x55,
 x56,
 x57,
 x58,
 x59,
 x60,
 x61,
 x62,
 x63,
 x64,
 x65,
 x66,
 x67,
 x68,
 x69,
 x70,
 x71,
 x72,
 x73,
 x74,
 x75,
 x76,
 x77,
 x78,
 x79,
 x80,
 x81,
 x82,
 x83,
 x84,
 x85,
 x86,
 x87,
 x88,
 x89,
 x90,
 x91,
 x92,
 x93,
 x94,
 x95,
 x96,
 x97,
 x98,
 x99,
 x100]

In [74]:
f_x = sum([xi**4 - 16*xi**2 + 5*xi for xi in variaveis])
print("Função 2:")
display(f_x)

Função 2:


x1**4 - 16*x1**2 + 5*x1 + x10**4 - 16*x10**2 + 5*x10 + x100**4 - 16*x100**2 + 5*x100 + x11**4 - 16*x11**2 + 5*x11 + x12**4 - 16*x12**2 + 5*x12 + x13**4 - 16*x13**2 + 5*x13 + x14**4 - 16*x14**2 + 5*x14 + x15**4 - 16*x15**2 + 5*x15 + x16**4 - 16*x16**2 + 5*x16 + x17**4 - 16*x17**2 + 5*x17 + x18**4 - 16*x18**2 + 5*x18 + x19**4 - 16*x19**2 + 5*x19 + x2**4 - 16*x2**2 + 5*x2 + x20**4 - 16*x20**2 + 5*x20 + x21**4 - 16*x21**2 + 5*x21 + x22**4 - 16*x22**2 + 5*x22 + x23**4 - 16*x23**2 + 5*x23 + x24**4 - 16*x24**2 + 5*x24 + x25**4 - 16*x25**2 + 5*x25 + x26**4 - 16*x26**2 + 5*x26 + x27**4 - 16*x27**2 + 5*x27 + x28**4 - 16*x28**2 + 5*x28 + x29**4 - 16*x29**2 + 5*x29 + x3**4 - 16*x3**2 + 5*x3 + x30**4 - 16*x30**2 + 5*x30 + x31**4 - 16*x31**2 + 5*x31 + x32**4 - 16*x32**2 + 5*x32 + x33**4 - 16*x33**2 + 5*x33 + x34**4 - 16*x34**2 + 5*x34 + x35**4 - 16*x35**2 + 5*x35 + x36**4 - 16*x36**2 + 5*x36 + x37**4 - 16*x37**2 + 5*x37 + x38**4 - 16*x38**2 + 5*x38 + x39**4 - 16*x39**2 + 5*x39 + x4**4 - 16*x4**2 + 5

In [75]:
grad_f_x = calcula_gradiente(f_x, variaveis)
print("Vetor gradiente da função 2:")
display(grad_f_x)

Vetor gradiente da função 2:


[4*x1**3 - 32*x1 + 5,
 4*x2**3 - 32*x2 + 5,
 4*x3**3 - 32*x3 + 5,
 4*x4**3 - 32*x4 + 5,
 4*x5**3 - 32*x5 + 5,
 4*x6**3 - 32*x6 + 5,
 4*x7**3 - 32*x7 + 5,
 4*x8**3 - 32*x8 + 5,
 4*x9**3 - 32*x9 + 5,
 4*x10**3 - 32*x10 + 5,
 4*x11**3 - 32*x11 + 5,
 4*x12**3 - 32*x12 + 5,
 4*x13**3 - 32*x13 + 5,
 4*x14**3 - 32*x14 + 5,
 4*x15**3 - 32*x15 + 5,
 4*x16**3 - 32*x16 + 5,
 4*x17**3 - 32*x17 + 5,
 4*x18**3 - 32*x18 + 5,
 4*x19**3 - 32*x19 + 5,
 4*x20**3 - 32*x20 + 5,
 4*x21**3 - 32*x21 + 5,
 4*x22**3 - 32*x22 + 5,
 4*x23**3 - 32*x23 + 5,
 4*x24**3 - 32*x24 + 5,
 4*x25**3 - 32*x25 + 5,
 4*x26**3 - 32*x26 + 5,
 4*x27**3 - 32*x27 + 5,
 4*x28**3 - 32*x28 + 5,
 4*x29**3 - 32*x29 + 5,
 4*x30**3 - 32*x30 + 5,
 4*x31**3 - 32*x31 + 5,
 4*x32**3 - 32*x32 + 5,
 4*x33**3 - 32*x33 + 5,
 4*x34**3 - 32*x34 + 5,
 4*x35**3 - 32*x35 + 5,
 4*x36**3 - 32*x36 + 5,
 4*x37**3 - 32*x37 + 5,
 4*x38**3 - 32*x38 + 5,
 4*x39**3 - 32*x39 + 5,
 4*x40**3 - 32*x40 + 5,
 4*x41**3 - 32*x41 + 5,
 4*x42**3 - 32*x42 + 5,
 4*x43**3 

In [76]:
hess_f_x = calcula_hessiana(f_x, variaveis)
print("Matriz Hessiana da função 2:")
display(hess_f_x)

Matriz Hessiana da função 2:


Matrix([
[12*x1**2 - 32,             0,             0,             0,             0,             0,             0,             0,             0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,              0,        

### Função 3

In [77]:
from sympy import symbols

variaveis = list(symbols('x_1 x_2'))
print("Variáveis da função 3:")
display(variaveis)

Variáveis da função 3:


[x_1, x_2]

In [78]:
x_1, x_2 = variaveis
f_x = (x_1**2 + x_2 - 11)**2 + (x_1 + x_2**2 - 7)**2
print("Função 3:")
display(f_x)

Função 3:


(x_1 + x_2**2 - 7)**2 + (x_1**2 + x_2 - 11)**2

In [79]:
grad_f_x = calcula_gradiente(f_x, variaveis)
print("Vetor gradiente da função 3:")
display(grad_f_x)

Vetor gradiente da função 3:


[4*x_1*(x_1**2 + x_2 - 11) + 2*x_1 + 2*x_2**2 - 14,
 2*x_1**2 + 4*x_2*(x_1 + x_2**2 - 7) + 2*x_2 - 22]

In [80]:
hess_f_x = calcula_hessiana(f_x, variaveis)
print("Matriz Hessiana da função 3:")
display(hess_f_x)

Matriz Hessiana da função 3:


Matrix([
[12*x_1**2 + 4*x_2 - 42,          4*x_1 + 4*x_2],
[         4*x_1 + 4*x_2, 4*x_1 + 12*x_2**2 - 26]])

In [ ]:
# TODO: Estudo da função: pontos críticos, convexidade, existência de ponto(s) ótimo(s), além de plotar sua função, etc.

In [ ]:
# TODO: Testar para 5 pontos distintos
ponto_inicial = np.array([5, 5])
d = np.array([1, 1])
eta = 0.02
gama = 0.08

passo_otimo, n_iteracoes = busca_de_armijo(f_x, grad_f_x, variaveis, ponto_inicial, d, gama, eta)
passo_otimo, n_iteracoes

(4.3980465111040014e-16, 15)

In [385]:
metodo_do_gradiente(f_x, grad_f_x, variaveis, ponto_inicial, gama, eta)

(array([2.99999999, 2.00000003]), 87, 261)

In [386]:
metodo_de_newton(f_x, grad_f_x, hess_f_x, variaveis, ponto_inicial, gama, eta)

(array([3., 2.]), 7, 7)

In [387]:
metodo_de_quase_newton(f_x, n_variaveis, grad_f_x, variaveis, ponto_inicial, gama, eta, metodo='bfgs')

(array([3., 2.]), 12, 16)